# Importing Libraries

In [1]:
import pandas as pd
import datetime

from varname import nameof
import time

# Functions for Calculating Technical Indicators

In [2]:
#function to get the stock name

def variable_to_str(df): 
    variable_name = list(df.keys())[1] 
    return variable_name[6:]

In [3]:
#Moving Average Convergence Divergence (MACD)

#https://aleksandarhaber.com/macd-moving-average-convergence-divergence-of-stock-price-time-series-in-pandas-and-python/


def MACD(data):
    
    short_period = 12
    long_period  = 26
    signal_period = 9

    stock_name = variable_to_str(df=data)
    
    data["ewm_short"] = data["Price_"+stock_name].ewm(span=short_period, adjust=False).mean()
    data["ewm_long"] = data["Price_"+stock_name].ewm(span=long_period, adjust=False).mean()
    data["MACD"] = data["ewm_short"] - data["ewm_long"]
    data["signal_MACD"] = data["MACD"].ewm(span=signal_period, adjust=False).mean()
    data["bars"] = data["MACD"] - data["signal_MACD"]
    
    data.drop(["ewm_short","ewm_long","signal_MACD","bars"],axis=1,inplace=True)
    
    data.rename(columns={"MACD":"MACD_"+stock_name},inplace=True)
    
    

In [4]:
#Relative Strength Index (RSI)

#https://www.youtube.com/watch?v=2H2Gd7zYVA8

def RSI(data):
    
    stock_name = variable_to_str(df=data)
    
    data["delta"] = data["Price_"+stock_name].diff()
    data["up"] = data["delta"].clip(lower=0)
    data["down"] = -1 * data["delta"].clip(upper=0)
    
    data["ema_up"] = data["up"].ewm(com=13, adjust=False).mean()
    data["ema_down"] = data["down"].ewm(com=13, adjust=False).mean()
    
    data["rs"] = data["ema_up"] / data["ema_down"]
    data["RSI"] = 100 - (100 / (1 + data["rs"]))
    
    data.drop(["delta","up","down","ema_up","ema_down","rs"], axis=1, inplace=True)
    data.rename(columns = {"RSI":"RSI_"+stock_name}, inplace=True)
    

In [5]:
#Commodity Channel Index (CCI)

#https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/cci
#https://gist.github.com/quantra-go-algo/1b37bfb74d69148f0dfbdb5a2c7bdb25

#TP = Typical Price
#sma = simple moving average
#mad = mean deviation

def CCI(data, number_of_days):
    
    stock_name = variable_to_str(df=data)
    
    data["TP"] = (data["High_"+stock_name] + data["Low_"+stock_name] + data["Price_"+stock_name])/3
    data['sma'] = data['TP'].rolling(number_of_days).mean()
    data['mad'] = data['TP'].rolling(number_of_days).apply(lambda x: pd.Series(x).mad())
    data['CCI'] = (data['TP'] - data['sma']) / (0.015 * data['mad']) 
    
    data.drop(["TP","sma", "mad"], axis=1, inplace=True)
    data.rename(columns = {"CCI":"CCI_"+stock_name}, inplace=True)


In [6]:
# Average Directional Index (ADX)

#https://medium.com/codex/algorithmic-trading-with-average-directional-index-in-python-2b5a20ecf06a

def calculateValue(df, column, index):
    previous_value = df.loc[index-1, column]
    current_value = df.loc[index, column]
    return current_value, previous_value

def calculateDM(df, index, stock_name):
    current_high, previous_high = calculateValue(df, "High_"+stock_name, index)
    current_low, previous_low = calculateValue(df, "Low_"+stock_name, index)
    
    dm_positive = current_high - previous_high
    dm_negative = current_low - previous_low
    
    if dm_positive > dm_negative:
        if dm_positive < 0:
            dm_positive = 0.00
        dm_negative = 0.00
        return dm_positive, dm_negative

    elif dm_positive < dm_negative:
        if dm_negative < 0:
            dm_negative = 0.00
        dm_positive = 0.00
        return dm_positive, dm_negative
    
    else:
        if dm_positive < 0:
            dm_positive = 0.00
        dm_negative = 0.00
        return dm_positive, dm_negative

def calculateTR(df, index, stock_name):
    current_high, previous_high = calculateValue(df, "High_"+stock_name, index)
    current_low, previous_low = calculateValue(df, "Low_"+stock_name, index)
    current_close, previous_close = calculateValue(df, "Price_"+stock_name, index)
    ranges = [current_high - current_low, abs(current_high - previous_close), abs(current_low - previous_close)]
    TR = max(ranges)
    return TR

def calculate_first_14(df, index, column):
    result = 0
    for i in range(index-13, index+1):
        result += df.loc[i, column]
    return result

def calculate_subsequent_14(df, index, column):
    return(df.loc[index-1, column+'14'] - (df.loc[index-1, column+'14']/14) + df.loc[index, column])


def calculate_first_adx(df, index):
    result = 0
    for i in range(index-13, index+1):
        result += df.loc[i, 'DX']
    return(result/14)

def calculate_adx(df, index):
    return(round(((df.loc[index-1, 'ADX']*13) + df.loc[index, 'DX'])/14, 2))

def ADX(data):
    
    stock_name = variable_to_str(df=data)
    
    for i in range(1, len(data)):
        dm_pos, dm_neg = calculateDM(data, i, stock_name)
        TR = calculateTR(data, i, stock_name)
        data.loc[i, '+DM'] = dm_pos
        data.loc[i, '-DM'] = dm_neg
        data.loc[i, 'TR'] = TR

        if data.TR.count() == 14:
            data.loc[i, 'TR14'] = calculate_first_14(data, i, 'TR')
            data.loc[i, '+DM14'] = calculate_first_14(data, i, '+DM')
            data.loc[i, '-DM14'] = calculate_first_14(data, i, '-DM')

        elif data.TR.count() >= 14:
            data.loc[i, 'TR14'] = round(calculate_subsequent_14(data, i, 'TR'),2)
            data.loc[i, '+DM14'] = round(calculate_subsequent_14(data, i, '+DM'), 2)
            data.loc[i, '-DM14'] = round(calculate_subsequent_14(data, i, '-DM'), 2)

        if 'TR14' in data.columns:
            data.loc[i, '+DI'] = round((data.loc[i, '+DM14'] / data.loc[i, 'TR14'])*100, 2)
            data.loc[i, '-DI'] = round((data.loc[i, '-DM14'] / data.loc[i, 'TR14'])*100, 2)

            data.loc[i, 'DX'] = round((abs(data.loc[i, '+DI'] - data.loc[i, '-DI'])/abs(data.loc[i, '+DI'] + data.loc[i, '-DI']) )*100 , 2)

        if 'DX' in data.columns:
            if data.DX.count() == 14:
                data.loc[i, 'ADX'] = calculate_first_adx(data, i)

            elif data.DX.count() >= 14:
                data.loc[i, 'ADX'] = calculate_adx(data, i)
    
    data.drop(["+DM","-DM","TR","TR14","+DM14","-DM14","+DI","-DI","DX"], axis=1, inplace=True)
    data.rename(columns={"ADX":"ADX_"+stock_name},inplace=True)


# GRAE - Grameenphone Ltd

In [24]:
GRAE = pd.read_csv("GRAE Historical Data.csv")
len_grae = len(GRAE)
print(len_grae)
GRAE.head()

3017


,Date,Price,Open,High,Low,Vol.,Change %
0,"Sep 04, 2022",288.7,295.0,295.8,288.0,121.84K,-0.45%
1,"Sep 01, 2022",290.0,290.0,291.0,289.0,161.03K,0.76%
2,"Aug 31, 2022",287.8,295.7,295.7,287.0,167.84K,-0.10%
3,"Aug 30, 2022",288.1,295.2,295.3,287.8,148.11K,-1.74%
4,"Aug 29, 2022",293.2,290.0,294.1,288.9,205.84K,1.52%


In [25]:
GRAE.rename(columns={"Price":"Price_GRAE","Open":"Open_GRAE","High":"High_GRAE","Low":"Low_GRAE","Vol.":"Vol_GRAE","Change %":"Change_GRAE"},inplace=True)
GRAE.columns

Index(['Date', 'Price_GRAE', 'Open_GRAE', 'High_GRAE', 'Low_GRAE', 'Vol_GRAE',
       'Change_GRAE'],
      dtype='object')

In [26]:
GRAE.dtypes

Date            object
Price_GRAE     float64
Open_GRAE      float64
High_GRAE      float64
Low_GRAE       float64
Vol_GRAE        object
Change_GRAE     object
dtype: object

In [27]:
GRAE["Date"] = pd.to_datetime(GRAE["Date"])
GRAE.dtypes

Date           datetime64[ns]
Price_GRAE            float64
Open_GRAE             float64
High_GRAE             float64
Low_GRAE              float64
Vol_GRAE               object
Change_GRAE            object
dtype: object

In [28]:
print(GRAE.isna().sum().sum())
GRAE[GRAE.isna().any(axis=1)]

2


,Date,Price_GRAE,Open_GRAE,High_GRAE,Low_GRAE,Vol_GRAE,Change_GRAE
809,2019-02-17,389.9,389.9,389.9,389.9,NaN,0.00%
939,2018-08-05,389.4,389.4,389.4,389.4,NaN,0.00%


In [29]:
GRAE.dropna(inplace=True)
GRAE.isna().sum().sum()
GRAE.reset_index(drop=True, inplace=True)

In [30]:
#Vol
GRAE["Vol_GRAE"] = GRAE["Vol_GRAE"].astype("str")
print(GRAE["Vol_GRAE"].apply(lambda x:x[-1:]).value_counts())
units = {'K':1e3, 'M':1e6}
GRAE["Vol_GRAE"] = GRAE["Vol_GRAE"].apply(lambda x:float(x[:-1])*units[x[-1:]])


#Change
GRAE["Change_GRAE"] = GRAE["Change_GRAE"].apply(lambda x:float(x[:-1]))

K    2698
M     317
Name: Vol_GRAE, dtype: int64


In [31]:
%%time

#calculating MACD
MACD(GRAE)

#calculating RSI
RSI(GRAE)

#calculating CCI
CCI(GRAE, number_of_days = 14)

#calculating RSI
ADX(GRAE)

Wall time: 22.7 s


In [32]:
GRAE.drop(["Open_GRAE", "High_GRAE", "Low_GRAE", "Vol_GRAE", "Change_GRAE"], axis=1, inplace=True)
GRAE.head()

,Date,Price_GRAE,MACD_GRAE,RSI_GRAE,CCI_GRAE,ADX_GRAE
0,2022-09-04,288.7,0.000000,NaN,NaN,NaN
1,2022-09-01,290.0,0.103704,100.000000,NaN,NaN
2,2022-08-31,287.8,0.008273,88.481675,NaN,NaN
3,2022-08-30,288.1,-0.042658,88.673267,NaN,NaN
4,2022-08-29,293.2,0.324763,91.317344,NaN,NaN


In [33]:
GRAE.isna().sum()

Date           0
Price_GRAE     0
MACD_GRAE      0
RSI_GRAE       1
CCI_GRAE      13
ADX_GRAE      27
dtype: int64

In [34]:
GRAE["RSI_GRAE"].bfill(limit=1, inplace=True)
GRAE["CCI_GRAE"].bfill(limit=13, inplace=True)
GRAE["ADX_GRAE"].bfill(limit=27, inplace=True)

In [35]:
GRAE.isna().sum()

Date          0
Price_GRAE    0
MACD_GRAE     0
RSI_GRAE      0
CCI_GRAE      0
ADX_GRAE      0
dtype: int64

In [36]:
GRAE.dtypes

Date          datetime64[ns]
Price_GRAE           float64
MACD_GRAE            float64
RSI_GRAE             float64
CCI_GRAE             float64
ADX_GRAE             float64
dtype: object

In [37]:
GRAE.describe()

,Price_GRAE,MACD_GRAE,RSI_GRAE,CCI_GRAE,ADX_GRAE
count,3015.000000,3015.000000,3015.000000,3015.000000,3015.000000
mean,288.928590,-0.235906,49.732764,5.267521,20.606776
std,84.898621,6.382620,13.913683,112.792073,9.368877
min,139.100000,-31.711179,14.643952,-348.025509,6.500000
25%,214.700000,-3.575598,39.283766,-85.646615,13.930000
50%,295.000000,-0.194910,49.407021,0.589226,18.500000
75%,349.750000,3.605784,60.621753,100.235440,24.430000
max,510.700000,18.718398,100.000000,466.666667,64.100000


# SQPH - Square Pharmaceuticals Ltd 

In [38]:
SQPH = pd.read_csv("SQPH Historical Data.csv")
len_sqph = len(SQPH)
print(len_sqph)
SQPH.head()

3504


,Date,Price,Open,High,Low,Vol.,Change %
0,"Sep 04, 2022",212.6,211.8,213.5,211.5,276.44K,0.38%
1,"Sep 01, 2022",211.8,213.0,213.0,211.4,1.02M,0.33%
2,"Aug 31, 2022",211.1,212.0,213.0,209.9,892.16K,-0.38%
3,"Aug 30, 2022",211.9,213.0,213.0,211.5,679.86K,-0.09%
4,"Aug 29, 2022",212.1,214.0,214.0,211.5,764.64K,-0.52%


In [39]:
SQPH.rename(columns={"Price":"Price_SQPH","Open":"Open_SQPH","High":"High_SQPH","Low":"Low_SQPH","Vol.":"Vol_SQPH","Change %":"Change_SQPH"},inplace=True)
SQPH.columns

Index(['Date', 'Price_SQPH', 'Open_SQPH', 'High_SQPH', 'Low_SQPH', 'Vol_SQPH',
       'Change_SQPH'],
      dtype='object')

In [40]:
SQPH.dtypes

Date            object
Price_SQPH     float64
Open_SQPH      float64
High_SQPH      float64
Low_SQPH       float64
Vol_SQPH        object
Change_SQPH     object
dtype: object

In [41]:
SQPH["Date"] = pd.to_datetime(SQPH["Date"])
SQPH.dtypes

Date           datetime64[ns]
Price_SQPH            float64
Open_SQPH             float64
High_SQPH             float64
Low_SQPH              float64
Vol_SQPH               object
Change_SQPH            object
dtype: object

In [42]:
print(SQPH.isna().sum().sum())
SQPH[SQPH.isna().any(axis=1)]

1


,Date,Price_SQPH,Open_SQPH,High_SQPH,Low_SQPH,Vol_SQPH,Change_SQPH
3417,2008-04-06,62.18,62.18,62.18,62.18,NaN,0.00%


In [43]:
SQPH.dropna(inplace=True)
print(SQPH.isna().sum().sum())
SQPH.reset_index(drop=True, inplace=True)

0


In [44]:
#Vol
SQPH["Vol_SQPH"] = SQPH["Vol_SQPH"].astype("str")
SQPH["Vol_SQPH"].apply(lambda x:x[-1:]).value_counts()
units = {'K':1e3, 'M':1e6}
SQPH["Vol_SQPH"] = SQPH["Vol_SQPH"].apply(lambda x:float(x[:-1])*units[x[-1:]])

#Change
SQPH["Change_SQPH"] = SQPH["Change_SQPH"].apply(lambda x:float(x[:-1]))

In [45]:
%%time

#calculating MACD
MACD(SQPH)

#calculating RSI
RSI(SQPH)

#calculating CCI
CCI(SQPH, number_of_days = 14)

#calculating RSI
ADX(SQPH)

Wall time: 26.3 s


In [46]:
SQPH.drop(["Open_SQPH", "High_SQPH", "Low_SQPH", "Vol_SQPH", "Change_SQPH"], axis=1, inplace=True)
SQPH.head()

,Date,Price_SQPH,MACD_SQPH,RSI_SQPH,CCI_SQPH,ADX_SQPH
0,2022-09-04,212.6,0.000000,NaN,NaN,NaN
1,2022-09-01,211.8,-0.063818,0.000000,NaN,NaN
2,2022-08-31,211.1,-0.168930,0.000000,NaN,NaN
3,2022-08-30,211.9,-0.185541,7.202572,NaN,NaN
4,2022-08-29,212.1,-0.180486,8.967826,NaN,NaN


In [47]:
SQPH.isna().sum()

Date           0
Price_SQPH     0
MACD_SQPH      0
RSI_SQPH       1
CCI_SQPH      18
ADX_SQPH      27
dtype: int64

In [53]:
SQPH["RSI_SQPH"].bfill(limit=1, inplace=True)
SQPH["CCI_SQPH"].bfill(limit=18, inplace=True)
SQPH["ADX_SQPH"].bfill(limit=27, inplace=True)

In [49]:
SQPH.isna().sum()

Date          0
Price_SQPH    0
MACD_SQPH     0
RSI_SQPH      0
CCI_SQPH      0
ADX_SQPH      0
dtype: int64

In [50]:
SQPH.head()

,Date,Price_SQPH,MACD_SQPH,RSI_SQPH,CCI_SQPH,ADX_SQPH
0,2022-09-04,212.6,0.000000,0.000000,95.847363,12.825714
1,2022-09-01,211.8,-0.063818,0.000000,95.847363,12.825714
2,2022-08-31,211.1,-0.168930,0.000000,95.847363,12.825714
3,2022-08-30,211.9,-0.185541,7.202572,95.847363,12.825714
4,2022-08-29,212.1,-0.180486,8.967826,95.847363,12.825714


In [51]:
SQPH.dtypes

Date          datetime64[ns]
Price_SQPH           float64
MACD_SQPH            float64
RSI_SQPH             float64
CCI_SQPH             float64
ADX_SQPH             float64
dtype: object

In [52]:
SQPH.describe()

,Price_SQPH,MACD_SQPH,RSI_SQPH,CCI_SQPH,ADX_SQPH
count,3503.000000,3503.000000,3503.000000,3503.000000,3503.000000
mean,149.694022,-0.328934,47.135280,inf,21.385435
std,72.999251,2.301539,13.792899,NaN,10.021925
min,39.450000,-9.014490,0.000000,-368.421053,2.730000
25%,76.065000,-1.515453,36.213161,-91.441800,13.810000
50%,168.610000,-0.321384,46.479479,-19.282736,19.440000
75%,215.550000,0.853173,57.537475,90.816562,26.270000
max,291.550000,8.482491,91.993521,inf,71.790000


# ACIF - ACI Formulations Ltd

In [54]:
ACIF = pd.read_csv("ACIF Historical Data.csv")
len_acif = len(ACIF)
print(len_acif)
ACIF.head()

3268


,Date,Price,Open,High,Low,Vol.,Change %
0,"Sep 04, 2022",173.8,183.0,184.0,171.6,831.80K,-0.57%
1,"Sep 01, 2022",174.8,174.4,175.8,170.9,551.36K,1.92%
2,"Aug 31, 2022",171.5,166.5,174.9,163.1,1.16M,4.38%
3,"Aug 30, 2022",164.3,163.0,166.2,162.8,304.96K,1.05%
4,"Aug 29, 2022",162.6,165.0,166.0,161.5,387.64K,-1.28%


In [55]:
ACIF.rename(columns={"Price":"Price_ACIF","Open":"Open_ACIF","High":"High_ACIF","Low":"Low_ACIF","Vol.":"Vol_ACIF","Change %":"Change_ACIF"},inplace=True)
ACIF.columns

Index(['Date', 'Price_ACIF', 'Open_ACIF', 'High_ACIF', 'Low_ACIF', 'Vol_ACIF',
       'Change_ACIF'],
      dtype='object')

In [56]:
ACIF.dtypes

Date            object
Price_ACIF     float64
Open_ACIF      float64
High_ACIF      float64
Low_ACIF       float64
Vol_ACIF        object
Change_ACIF     object
dtype: object

In [57]:
ACIF["Date"] = pd.to_datetime(ACIF["Date"])
ACIF.dtypes

Date           datetime64[ns]
Price_ACIF            float64
Open_ACIF             float64
High_ACIF             float64
Low_ACIF              float64
Vol_ACIF               object
Change_ACIF            object
dtype: object

In [58]:
print(ACIF.isna().sum().sum())

0


In [59]:
#Vol
ACIF["Vol_ACIF"] = ACIF["Vol_ACIF"].astype("str")
ACIF["Vol_ACIF"].apply(lambda x:x[-1:]).value_counts()
units = {'K':1e3, 'M':1e6}
ACIF["Vol_ACIF"] = ACIF["Vol_ACIF"].apply(lambda x:float(x[:-1])*units[x[-1:]])

#Change
ACIF["Change_ACIF"] = ACIF["Change_ACIF"].apply(lambda x:float(x[:-1]))

In [60]:
%%time

#calculating MACD
MACD(ACIF)

#calculating RSI
RSI(ACIF)

#calculating CCI
CCI(ACIF, number_of_days = 14)

#calculating RSI
ADX(ACIF)

Wall time: 24.4 s


In [61]:
ACIF.drop(["Open_ACIF", "High_ACIF", "Low_ACIF", "Vol_ACIF", "Change_ACIF"], axis=1, inplace=True)
ACIF.head()

,Date,Price_ACIF,MACD_ACIF,RSI_ACIF,CCI_ACIF,ADX_ACIF
0,2022-09-04,173.8,0.000000,NaN,NaN,NaN
1,2022-09-01,174.8,0.079772,100.000000,NaN,NaN
2,2022-08-31,171.5,-0.121885,79.754601,NaN,NaN
3,2022-08-30,164.3,-0.852849,54.045411,NaN,NaN
4,2022-08-29,162.6,-1.551435,49.951117,NaN,NaN


In [62]:
ACIF.isna().sum()

Date           0
Price_ACIF     0
MACD_ACIF      0
RSI_ACIF       1
CCI_ACIF      13
ADX_ACIF      27
dtype: int64

In [64]:
ACIF["RSI_ACIF"].bfill(limit=1, inplace=True)
ACIF["CCI_ACIF"].bfill(limit=18, inplace=True)
ACIF["ADX_ACIF"].bfill(limit=27, inplace=True)

In [65]:
ACIF.isna().sum()

Date          0
Price_ACIF    0
MACD_ACIF     0
RSI_ACIF      0
CCI_ACIF      0
ADX_ACIF      0
dtype: int64

In [66]:
ACIF.head()

,Date,Price_ACIF,MACD_ACIF,RSI_ACIF,CCI_ACIF,ADX_ACIF
0,2022-09-04,173.8,0.000000,100.000000,-92.565445,62.418571
1,2022-09-01,174.8,0.079772,100.000000,-92.565445,62.418571
2,2022-08-31,171.5,-0.121885,79.754601,-92.565445,62.418571
3,2022-08-30,164.3,-0.852849,54.045411,-92.565445,62.418571
4,2022-08-29,162.6,-1.551435,49.951117,-92.565445,62.418571


In [67]:
ACIF.dtypes

Date          datetime64[ns]
Price_ACIF           float64
MACD_ACIF            float64
RSI_ACIF             float64
CCI_ACIF             float64
ADX_ACIF             float64
dtype: object

In [68]:
ACIF.describe()

,Price_ACIF,MACD_ACIF,RSI_ACIF,CCI_ACIF,ADX_ACIF
count,3268.000000,3268.000000,3268.000000,3268.000000,3268.000000
mean,124.084819,-0.152888,50.686797,11.516218,23.195236
std,41.476406,3.412174,11.694522,108.587760,11.817574
min,40.320000,-20.387251,16.714488,-355.668590,6.750000
25%,84.760000,-1.561193,42.076843,-74.856811,14.500000
50%,119.110000,0.088673,50.705780,14.075325,19.870000
75%,157.062500,1.828604,59.273017,98.993122,28.440000
max,242.860000,11.970121,100.000000,340.761461,64.030000


# Merge - 3 Stocks

In [69]:
print(min(len_grae, len_sqph, len_acif))
print(max(len_grae, len_sqph, len_acif))

3017
3504


In [70]:
data = pd.merge(pd.merge(GRAE,SQPH,on='Date'),ACIF,on='Date')
print(len(data))
data.head()

2989


,Date,Price_GRAE,MACD_GRAE,RSI_GRAE,CCI_GRAE,ADX_GRAE,Price_SQPH,MACD_SQPH,RSI_SQPH,CCI_SQPH,ADX_SQPH,Price_ACIF,MACD_ACIF,RSI_ACIF,CCI_ACIF,ADX_ACIF
0,2022-09-04,288.7,0.000000,100.000000,-14.475309,34.675,212.6,0.000000,0.000000,95.847363,12.825714,173.8,0.000000,100.000000,-92.565445,62.418571
1,2022-09-01,290.0,0.103704,100.000000,-14.475309,34.675,211.8,-0.063818,0.000000,95.847363,12.825714,174.8,0.079772,100.000000,-92.565445,62.418571
2,2022-08-31,287.8,0.008273,88.481675,-14.475309,34.675,211.1,-0.168930,0.000000,95.847363,12.825714,171.5,-0.121885,79.754601,-92.565445,62.418571
3,2022-08-30,288.1,-0.042658,88.673267,-14.475309,34.675,211.9,-0.185541,7.202572,95.847363,12.825714,164.3,-0.852849,54.045411,-92.565445,62.418571
4,2022-08-29,293.2,0.324763,91.317344,-14.475309,34.675,212.1,-0.180486,8.967826,95.847363,12.825714,162.6,-1.551435,49.951117,-92.565445,62.418571


In [71]:
data.tail()

,Date,Price_GRAE,MACD_GRAE,RSI_GRAE,CCI_GRAE,ADX_GRAE,Price_SQPH,MACD_SQPH,RSI_SQPH,CCI_SQPH,ADX_SQPH,Price_ACIF,MACD_ACIF,RSI_ACIF,CCI_ACIF,ADX_ACIF
2984,2009-11-24,174.8,-9.243747,36.283072,-11.913146,45.18,52.81,-0.609071,45.580093,34.504005,18.87,112.57,-0.106855,55.499371,117.189982,10.08
2985,2009-11-23,168.5,-9.150214,31.494288,-131.861199,46.38,51.80,-0.624922,37.973017,-123.819554,17.91,110.29,-0.147102,47.698214,-14.701006,10.16
2986,2009-11-22,165.7,-9.196020,29.622953,-137.570621,47.49,50.46,-0.737113,30.661547,-207.836308,17.01,110.60,-0.152228,48.752968,-7.592998,10.18
2987,2009-11-19,164.0,-9.262722,28.515129,-178.234488,48.52,50.56,-0.808635,31.718157,-166.432532,16.14,109.65,-0.230293,45.710690,-68.681733,10.19
2988,2009-11-18,158.2,-9.672102,25.070313,-137.446000,46.17,50.47,-0.862635,31.256511,-141.855707,15.36,109.71,-0.284044,45.940138,-49.687199,9.68


In [72]:
data.columns

Index(['Date', 'Price_GRAE', 'MACD_GRAE', 'RSI_GRAE', 'CCI_GRAE', 'ADX_GRAE',
       'Price_SQPH', 'MACD_SQPH', 'RSI_SQPH', 'CCI_SQPH', 'ADX_SQPH',
       'Price_ACIF', 'MACD_ACIF', 'RSI_ACIF', 'CCI_ACIF', 'ADX_ACIF'],
      dtype='object')

In [73]:
data.isna().sum().sum()

0

In [74]:
data.to_csv("stock_data.csv",index=False)